In [1]:
#https://jsdokken.com/dolfinx-tutorial/chapter2/navierstokes.html#variational-formulation

from dolfinx import mesh, fem, io
import ufl
from mpi4py import MPI
from petsc4py import PETSc
import numpy as np

T = 5
num_steps = 1000
n_cells = 32

In [2]:
domain = mesh.create_unit_square(MPI.COMM_WORLD, n_cells, n_cells)
velocity_function_space = fem.functionspace(domain, ("Lagrange", 2, (2,))) 
pressure_function_space = fem.functionspace(domain, ("Lagrange", 1))

In [3]:
f = fem.Constant(domain, PETSc.ScalarType((0,0)))
dt = fem.Constant(domain, T/num_steps)
mu = fem.Constant(domain, PETSc.ScalarType(1))
rho = fem.Constant(domain, PETSc.ScalarType(1))

In [4]:
u = ufl.TrialFunction(velocity_function_space)
p = ufl.TrialFunction(pressure_function_space)

v = ufl.TestFunction(velocity_function_space)
q = ufl.TestFunction(pressure_function_space)

u_sol = fem.Function(velocity_function_space) # function to store u solved
u_prev = fem.Function(velocity_function_space) # u from previous time step
p_sol = fem.Function(pressure_function_space) 
p_prev = fem.Function(pressure_function_space)

FunctionSpace(Mesh(blocked element (Basix element (P, triangle, 1, gll_warped, unset, False, float64, []), (2,)), 0), blocked element (Basix element (P, triangle, 2, gll_warped, unset, False, float64, []), (2,)))

In [5]:
# poisellieu flow
def inflow(x):
    return np.isclose(x[0], 0)

def outflow(x):
    return np.isclose(x[0], 1)

def walls(x):
    return np.logical_or(
        np.isclose(x[1], 0), np.isclose(x[1], 1)
   )

fdim = domain.topology.dim - 1
inflow_facets = mesh.locate_entities_boundary(domain, fdim, inflow)
dofs_inflow = fem.locate_dofs_topological(pressure_function_space, fdim, inflow_facets)
bc_inflow  = fem.dirichletbc(fem.Constant(domain, PETSc.ScalarType(8)), dofs_inflow, pressure_function_space)

outflow_facets = mesh.locate_entities_boundary(domain, fdim, outflow)
dofs_outflow = fem.locate_dofs_topological(pressure_function_space, fdim, outflow_facets)
bc_outflow  = fem.dirichletbc(fem.Constant(domain, PETSc.ScalarType(0)), dofs_outflow, pressure_function_space)
bc_p = [bc_inflow, bc_outflow]

walls_facets = mesh.locate_entities_boundary(domain, fdim, walls)
dofs_walls = fem.locate_dofs_topological(velocity_function_space, fdim, walls_facets)
bc_noslip  = fem.dirichletbc(fem.Constant(domain, PETSc.ScalarType((0, 0))), dofs_walls, velocity_function_space)
bc_u = [bc_noslip]

In [6]:
from ufl import FacetNormal, dx, ds, dot, inner, sym, nabla_grad, Identity, lhs, rhs, div

u_midpoint = 0.5*(u_prev + u)
n = FacetNormal(domain)

def epsilon(u):
    return sym(nabla_grad(u))

def sigma(u, p):
    return 2*mu*epsilon(u) - p*Identity(len(u))

# step 1
form1 = rho*dot((u - u_prev) / dt, v)*dx \
      + rho*dot(dot(u_prev, nabla_grad(u_prev)), v)*dx \
      + inner(sigma(u_midpoint, p_prev), epsilon(v))*dx \
      + dot(p_prev*n, v)*ds - dot(mu*nabla_grad(u_midpoint)*n, v)*ds \
      - dot(f, v)*dx
bilinear1 = lhs(form1)
linear1 = rhs(form1)

# bilinear1 is not time dependent, it can be assembled only once
A1 = fem.assemble_matrix(bilinear1, bcs=bc_u)
A1.assemble()

#linear1 is time dependent
b1 = create_vector(linear1) # 

# step 2
form2 = dot(nabla_grad(p), nabla_grad(q))*dx \
      - dot(nabla_grad(p_prev), nabla_grad(q))*dx \
      + (rho/dt)*div(u_sol)*q*dx
bilinear2 = lhs(form2)
linear2 = rhs(form2)

# step 3
form3 = rho*dot((u - u_sol), v)*dx \
      + dt*dot(nabla_grad(p_sol - p_prev), v)*dx
bilinear3 = lhs(form3)
linear3 = rhs(form3)

In [7]:
from dolfinx.fem.petsc import LinearProblem

t = 0
u_file = io.VTXWriter(domain.comm, "unit_square1_10_03/u.bp", u_sol)
p_file = io.VTXWriter(domain.comm, "unit_square1_10_03/p.bp", p_sol)
u_file.write(t)
p_file.write(t)

for n in range(num_steps):
    t += dt
    
    problem1 = LinearProblem(bilinear1, linear1, bc_u, u_sol)
    problem1.solve()

    problem2 = LinearProblem(bilinear2, linear2, bc_p, p_sol)
    problem2.solve()

    problem3 = LinearProblem(bilinear3, linear3, bc_u, u_sol)
    problem3.solve()

    u_file.write(t)
    p_file.write(t)
    
    # compute error
    u_e = fem.Function(velocity_function_space)
    u_e.interpolate(lambda x: np.vstack((4.0*x[1]*(1.0 - x[1]), 0.0*x[0])))
    error = np.abs(u_e.x.array - u_sol.x.array).max()
    print('t = %.2f: error = %.3g' % (t, error))

    u_prev.x.array[:] = u_sol.x.array
    p_prev.x.array[:] = p_sol.x.array

t = 0.01: error = 0.96
t = 0.01: error = 0.92
t = 0.01: error = 0.88
t = 0.02: error = 0.841
t = 0.03: error = 0.802
t = 0.03: error = 0.764
t = 0.04: error = 0.728
t = 0.04: error = 0.694
t = 0.04: error = 0.66
t = 0.05: error = 0.629
t = 0.05: error = 0.599
t = 0.06: error = 0.57
t = 0.06: error = 0.543
t = 0.07: error = 0.516
t = 0.07: error = 0.492
t = 0.08: error = 0.468
t = 0.09: error = 0.445
t = 0.09: error = 0.424
t = 0.10: error = 0.404
t = 0.10: error = 0.384
t = 0.11: error = 0.366
t = 0.11: error = 0.348
t = 0.12: error = 0.331
t = 0.12: error = 0.315
t = 0.13: error = 0.3
t = 0.13: error = 0.286
t = 0.14: error = 0.272
t = 0.14: error = 0.259
t = 0.15: error = 0.246
t = 0.15: error = 0.234
t = 0.16: error = 0.223
t = 0.16: error = 0.212
t = 0.17: error = 0.202
t = 0.17: error = 0.192
t = 0.18: error = 0.183
t = 0.18: error = 0.174
t = 0.19: error = 0.166
t = 0.19: error = 0.158
t = 0.20: error = 0.15
t = 0.20: error = 0.143
t = 0.21: error = 0.136
t = 0.21: error = 0.13
t